In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install openl3

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327030 sha256=ce9e2a83d621267b04ce0b638ca1ea3b12c20007c6828a6a0361fbcabc1abcef
  Stored in directory: /root/.cache/pip/wheels/35/e9/4c/b1e39385b21f2b4d70c01b8793ecc921d69f167cc772868abe
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29642 sha256=da5aea0ced9312c23969b3b0ec6cf9e3797e344baaf23ad78fac7054cd89cd2e
  Stored in directory: /root/.cache/pip/wheels/d2/17/ce/09b7d799a7c8b463b0a1657941331e0f10112864ff78d28f87
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320705 sha256=a251d064d67ef0ca59fa2d9eebdbea1516598c79f3501d22d3c0e925e314be8f
  Stored in directory: /root/

In [ ]:
!pip install joblib

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import librosa
import openl3
import pickle
import joblib
import json
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

class UnifiedContrastiveModel(nn.Module):
    """
    Your exact model from training code - creates unified embeddings
    """
    def __init__(self, symbolic_dim=768, audio_dim=6144, latent_dim=512):
        super().__init__()

        # Individual encoders
        self.symbolic_encoder = self._build_encoder(symbolic_dim, latent_dim // 2)
        self.audio_encoder = self._build_encoder(audio_dim, latent_dim // 2)

        # Fusion layer for unified embedding
        self.fusion_layer = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(latent_dim, latent_dim)
        )

        # Temperature parameter for contrastive learning
        self.temperature = nn.Parameter(torch.tensor(0.07))

    def _build_encoder(self, input_dim: int, output_dim: int) -> nn.Module:
        """Build encoder network"""
        hidden_dim = max(512, input_dim // 4)
        return nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, output_dim)
        )

    def forward(self, symbolic_features, audio_features):
        # Ensure proper shapes
        if symbolic_features.dim() == 3:
            symbolic_features = symbolic_features.squeeze(1)
        if audio_features.dim() == 3:
            audio_features = audio_features.squeeze(1)

        # Encode individual modalities
        symbolic_emb = self.symbolic_encoder(symbolic_features)
        audio_emb = self.audio_encoder(audio_features)

        # Concatenate and create unified embedding
        combined = torch.cat([symbolic_emb, audio_emb], dim=1)
        unified_embedding = self.fusion_layer(combined)

        # Normalize the unified embedding
        unified_embedding = F.normalize(unified_embedding, p=2, dim=1)

        return unified_embedding

class MusicPlagiarismChecker:
    def __init__(self, unified_model_path, reference_db_path, debug=False):
        """
        Initialize the plagiarism checker
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.debug = debug

        # Load the OpenL3 model for feature extraction
        if not debug:
            print("Loading OpenL3 model...")
        self.openl3_model = openl3.models.load_audio_embedding_model(
            input_repr="mel256",
            content_type="music",
            embedding_size=6144
        )

        # Load your unified contrastive model
        if not debug:
            print("Loading unified contrastive model...")
        self.unified_model = UnifiedContrastiveModel()

        try:
            checkpoint = torch.load(unified_model_path, map_location=self.device)
            if 'model_state_dict' in checkpoint:
                self.unified_model.load_state_dict(checkpoint['model_state_dict'])
            else:
                self.unified_model.load_state_dict(checkpoint)
        except Exception as e:
            print(f"Error loading model: {e}")
            self.unified_model = torch.load(unified_model_path, map_location=self.device)

        self.unified_model.eval()
        self.unified_model.to(self.device)

        # Load reference database
        if not debug:
            print("Loading reference database...")
        self.reference_data = joblib.load(reference_db_path)
        self._prepare_reference_data()

        # Adjusted thresholds for audio-only matching
        self.thresholds = {
            'HIGH': 0.50,     # Adjusted based on your Bach→Schubert result (0.356)
            'MEDIUM': 0.35,   # Set below your current result
            'LOW': 0.25,      # Conservative threshold
            'NONE': 0.0
        }

    def _prepare_reference_data(self):
        """Prepare reference embeddings and metadata for comparison"""
        self.ref_embeddings = []
        self.ref_metadata = []

        if 'embeddings' in self.reference_data and 'metadata' in self.reference_data:
            embeddings = self.reference_data['embeddings']
            metadata = self.reference_data['metadata']

            for piece_id in embeddings.keys():
                embedding = embeddings[piece_id]
                if isinstance(embedding, np.ndarray):
                    self.ref_embeddings.append(embedding)
                else:
                    self.ref_embeddings.append(np.array(embedding))

                meta = metadata.get(piece_id, {})
                meta['piece_id'] = piece_id
                self.ref_metadata.append(meta)

        if self.ref_embeddings:
            self.ref_embeddings = np.array(self.ref_embeddings)
            if not self.debug:
                print(f"✅ Loaded {len(self.ref_embeddings)} reference embeddings")

    def extract_symbolic_features_from_audio(self, audio, sr):
        """
        Fixed: Extract symbolic-like features from audio using signal processing
        """
        try:
            features = []

            # 1. Chroma features (12-dimensional)
            chroma = librosa.feature.chroma_stft(y=audio, sr=sr, n_fft=2048, hop_length=512)
            chroma_mean = np.mean(chroma, axis=1)
            chroma_std = np.std(chroma, axis=1)
            features.extend(chroma_mean.tolist())  # Convert to list to avoid array issues
            features.extend(chroma_std.tolist())   # 24 features total

            # 2. Tonnetz (6-dimensional)
            tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
            tonnetz_mean = np.mean(tonnetz, axis=1)
            tonnetz_std = np.std(tonnetz, axis=1)
            features.extend(tonnetz_mean.tolist())  # 6 features
            features.extend(tonnetz_std.tolist())   # 6 more features

            # 3. Tempo and rhythmic features
            try:
                tempo, beats = librosa.beat.beat_track(y=audio, sr=sr)
                features.append(float(tempo) / 200.0)  # Normalize tempo
                # Beat strength
                if len(beats) > 1:
                    beat_intervals = np.diff(beats) / sr
                    features.append(float(np.mean(beat_intervals)))
                    features.append(float(np.std(beat_intervals)))
                else:
                    features.extend([0.5, 0.1])  # Default values
            except:
                features.extend([0.6, 0.5, 0.1])  # Default tempo and beat features

            # 4. Spectral features
            try:
                spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)
                spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)
                spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)

                features.append(float(np.mean(spectral_centroids)) / 4000.0)
                features.append(float(np.std(spectral_centroids)) / 4000.0)
                features.append(float(np.mean(spectral_rolloff)) / 4000.0)
                features.append(float(np.std(spectral_rolloff)) / 4000.0)
                features.append(float(np.mean(spectral_bandwidth)) / 2000.0)
                features.append(float(np.std(spectral_bandwidth)) / 2000.0)
            except:
                features.extend([0.5, 0.1, 0.6, 0.1, 0.4, 0.1])  # Default values

            # 5. Zero crossing rate
            try:
                zcr = librosa.feature.zero_crossing_rate(audio)
                features.append(float(np.mean(zcr)))
                features.append(float(np.std(zcr)))
            except:
                features.extend([0.1, 0.02])

            # 6. MFCC features (first 13 coefficients)
            try:
                mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
                mfcc_mean = np.mean(mfccs, axis=1)
                mfcc_std = np.std(mfccs, axis=1)
                features.extend(mfcc_mean.tolist())  # 13 features
                features.extend(mfcc_std.tolist())   # 13 more features
            except:
                features.extend([0.0] * 26)  # Default MFCC features

            # Convert to numpy array and ensure we have the right dimension
            current_features = np.array(features, dtype=np.float32)
            target_dim = 768

            # Pad or truncate to match expected dimension
            if len(current_features) < target_dim:
                # Repeat the pattern to fill the dimension
                repeat_count = target_dim // len(current_features) + 1
                padded = np.tile(current_features, repeat_count)[:target_dim]
                return padded.astype(np.float32)
            else:
                return current_features[:target_dim].astype(np.float32)

        except Exception as e:
            if self.debug:
                print(f"Error extracting symbolic features from audio: {e}")
            # Return zeros as fallback
            return np.zeros(768, dtype=np.float32)

    def segment_audio(self, audio, sr, segment_length=10.0, stride=5.0):
        """Segment audio into overlapping chunks"""
        segments = []
        metadata = []
        total_duration = len(audio) / sr
        start = 0.0
        segment_id = 0

        while start < total_duration:
            end = start + segment_length
            start_sample = int(start * sr)
            end_sample = int(end * sr)

            if end_sample > len(audio):
                segment = np.pad(audio[start_sample:], (0, end_sample - len(audio)))
                is_padded = True
            else:
                segment = audio[start_sample:end_sample]
                is_padded = False

            segments.append(segment)
            metadata.append({
                "segment_id": segment_id,
                "start_time": float(start),
                "end_time": float(min(end, total_duration)),
                "duration": segment_length,
                "is_padded": is_padded,
            })

            start += stride
            segment_id += 1

        return segments, metadata

    def extract_features(self, audio_path):
        """Extract both audio and pseudo-symbolic features"""
        try:
            # Load audio
            audio, sr = librosa.load(audio_path, sr=48000)

            # Segment audio
            segments, seg_metadata = self.segment_audio(audio, sr)

            audio_embeddings = []
            symbolic_features = []

            for segment in segments:
                # Extract OpenL3 audio embedding
                audio_emb, _ = openl3.get_audio_embedding(
                    segment,
                    sr,
                    model=self.openl3_model,
                    center=False,
                    hop_size=1.0,
                    verbose=False
                )
                # Aggregate embeddings (mean pooling)
                aggregated_audio_emb = np.mean(audio_emb, axis=0)
                audio_embeddings.append(aggregated_audio_emb)

                # Extract pseudo-symbolic features from the same segment
                symbolic_feat = self.extract_symbolic_features_from_audio(segment, sr)
                symbolic_features.append(symbolic_feat)

            return np.array(audio_embeddings), np.array(symbolic_features), seg_metadata

        except Exception as e:
            print(f"Error processing {audio_path}: {str(e)}")
            return None, None, None

    def get_unified_embeddings(self, audio_embeddings, symbolic_features):
        """Get unified embeddings using multiple approaches like in debug mode"""
        approaches = {}

        with torch.no_grad():
            audio_tensor = torch.FloatTensor(audio_embeddings).to(self.device)
            symbolic_tensor = torch.FloatTensor(symbolic_features).to(self.device)
            batch_size = audio_tensor.shape[0]

            # Approach 1: Use pseudo-symbolic features
            try:
                unified_pseudo = self.unified_model(symbolic_tensor, audio_tensor)
                approaches['pseudo_symbolic'] = unified_pseudo.cpu().numpy()
            except:
                pass

            # Approach 2: Zero symbolic (this was working in debug!)
            dummy_symbolic = torch.zeros(batch_size, 768, device=self.device)
            unified_zero = self.unified_model(dummy_symbolic, audio_tensor)
            approaches['zero_symbolic'] = unified_zero.cpu().numpy()

            # Approach 3: Scaled symbolic features
            try:
                scaled_symbolic = symbolic_tensor * 0.1
                unified_scaled = self.unified_model(scaled_symbolic, audio_tensor)
                approaches['scaled_symbolic'] = unified_scaled.cpu().numpy()
            except:
                pass

        # Return the approach that gives best similarity scores
        # For now, prioritize zero_symbolic since it was working
        if 'zero_symbolic' in approaches:
            return approaches['zero_symbolic']
        elif 'pseudo_symbolic' in approaches:
            return approaches['pseudo_symbolic']
        else:
            return approaches['scaled_symbolic']

    def find_matches(self, query_embeddings, query_metadata, similarity_threshold=0.15):
        """Find matches using cosine similarity with multiple attempts"""
        if len(self.ref_embeddings) == 0:
            return []

        matches = []

        try:
            # Compute similarity matrix
            similarity_matrix = cosine_similarity(query_embeddings, self.ref_embeddings)

            # Debug info for first file
            if self.debug:
                print(f"Similarity matrix shape: {similarity_matrix.shape}")
                print(f"Max similarity: {np.max(similarity_matrix):.4f}")
                print(f"Mean similarity: {np.mean(similarity_matrix):.4f}")

            # Find matches for each query segment
            for q_idx, query_scores in enumerate(similarity_matrix):
                max_score_idx = np.argmax(query_scores)
                max_score = query_scores[max_score_idx]

                # Try multiple thresholds if no matches found
                thresholds_to_try = [similarity_threshold, 0.1, 0.05, 0.01]

                for thresh in thresholds_to_try:
                    if max_score >= thresh:
                        ref_meta = self.ref_metadata[max_score_idx]
                        query_meta = query_metadata[q_idx]

                        matches.append({
                            'query_segment': q_idx,
                            'query_time': (query_meta['start_time'], query_meta['end_time']),
                            'reference_work': ref_meta.get('composition', ref_meta.get('piece_id', 'Unknown')),
                            'reference_composer': ref_meta.get('composer', 'Unknown'),
                            'reference_piece_id': ref_meta.get('piece_id', 'Unknown'),
                            'similarity_score': float(max_score)
                        })
                        break  # Found match, no need to try lower thresholds

        except Exception as e:
            print(f"Error computing similarities: {e}")

        return matches

    def assess_plagiarism_likelihood(self, matches, total_segments):
        """Assess plagiarism likelihood with adjusted thresholds"""
        if not matches:
            return 'NONE', 0.0

        match_percentage = len(matches) / total_segments * 100
        avg_score = np.mean([m['similarity_score'] for m in matches])
        max_score = max([m['similarity_score'] for m in matches])

        # Assessment logic based on your debug results
        if avg_score >= self.thresholds['HIGH'] and match_percentage >= 30:
            return 'HIGH', min(0.90, avg_score)
        elif avg_score >= self.thresholds['MEDIUM'] and match_percentage >= 20:
            return 'MEDIUM', min(0.75, avg_score)
        elif avg_score >= self.thresholds['LOW'] or max_score >= self.thresholds['MEDIUM']:
            return 'LOW', min(0.60, max(avg_score, max_score * 0.8))
        else:
            return 'NONE', min(0.50, avg_score)

    def group_matches_by_work(self, matches):
        """Group matches by reference work"""
        work_matches = defaultdict(list)

        for match in matches:
            work_key = f"{match['reference_work']} - {match['reference_composer']}"
            work_matches[work_key].append(match)

        grouped_results = []
        for work_key, work_match_list in work_matches.items():
            try:
                work_title, composer = work_key.split(' - ', 1)
            except ValueError:
                work_title, composer = work_key, 'Unknown'

            total_duration = len(work_match_list) * 10.0
            avg_score = np.mean([m['similarity_score'] for m in work_match_list])

            grouped_results.append({
                'work': work_title,
                'composer': composer,
                'total_matched_duration': total_duration,
                'average_similarity': avg_score,
                'matched_segments': [
                    {
                        'query_time': match['query_time'],
                        'reference_time': (0, 10),
                        'score': match['similarity_score']
                    }
                    for match in sorted(work_match_list, key=lambda x: x['similarity_score'], reverse=True)
                ]
            })

        return sorted(grouped_results, key=lambda x: x['average_similarity'], reverse=True)

    def analyze_single_file(self, audio_path):
        """Analyze a single audio file for plagiarism"""
        filename = os.path.basename(audio_path)

        # Extract both audio and symbolic features
        audio_embeddings, symbolic_features, metadata = self.extract_features(audio_path)
        if audio_embeddings is None:
            return None

        # Convert to unified embeddings
        unified_embeddings = self.get_unified_embeddings(audio_embeddings, symbolic_features)

        # Find matches with very low threshold initially
        matches = self.find_matches(unified_embeddings, metadata, similarity_threshold=0.2)

        # If no matches with relaxed threshold, report the max similarity anyway
        if not matches:
            try:
                similarity_matrix = cosine_similarity(unified_embeddings, self.ref_embeddings)
                max_similarity = np.max(similarity_matrix)
                print(f"   No matches found. Max similarity: {max_similarity:.4f}")
            except:
                pass

        # Assess plagiarism
        likelihood, confidence = self.assess_plagiarism_likelihood(matches, len(metadata))
        grouped_matches = self.group_matches_by_work(matches)

        avg_score = np.mean([m['similarity_score'] for m in matches]) if matches else 0.0
        max_score = max([m['similarity_score'] for m in matches]) if matches else 0.0
        match_percentage = (len(matches) / len(metadata)) * 100

        result = {
            'query_song': filename,
            'analysis_summary': {
                'segments_analyzed': len(metadata),
                'segments_with_matches': len(matches),
                'percentage_matched': f"{match_percentage:.1f}%",
                'highest_match_score': f"{max_score:.3f}",
                'average_match_score': f"{avg_score:.3f}",
                'plagiarism_assessment': {
                    'likelihood': likelihood,
                    'confidence': f"{confidence:.2f}"
                }
            },
            'matched_works': grouped_matches
        }

        return result

    def analyze_batch(self, test_directory, output_file=None):
        """Analyze multiple files in a directory"""
        results = []

        # Find all audio files
        audio_files = []
        for root, dirs, files in os.walk(test_directory):
            for file in files:
                if file.lower().endswith(('.wav', '.mp3', '.flac', '.m4a')):
                    audio_files.append(os.path.join(root, file))

        if not audio_files:
            print("No audio files found in the specified directory.")
            return []

        print(f"Found {len(audio_files)} audio files to analyze...")

        # Analyze each file with progress bar
        for audio_file in tqdm(audio_files, desc="Analyzing files"):
            try:
                result = self.analyze_single_file(audio_file)
                if result:
                    results.append(result)

                    # Print summary for each file
                    print(f"\n📄 {result['query_song']}")
                    print(f"   Assessment: {result['analysis_summary']['plagiarism_assessment']['likelihood']}")
                    print(f"   Confidence: {result['analysis_summary']['plagiarism_assessment']['confidence']}")
                    print(f"   Matches: {result['analysis_summary']['segments_with_matches']}/{result['analysis_summary']['segments_analyzed']} segments")
                    if result['matched_works']:
                        print(f"   Top match: {result['matched_works'][0]['work']} by {result['matched_works'][0]['composer']}")

            except Exception as e:
                print(f"Error analyzing {os.path.basename(audio_file)}: {e}")

        # Save results if requested
        if output_file:
            with open(output_file, 'w') as f:
                json.dump(results, f, indent=2)
            print(f"\nResults saved to {output_file}")

        # Print summary statistics
        self.print_batch_summary(results)

        return results

    def print_batch_summary(self, results):
        """Print summary statistics for batch analysis"""
        if not results:
            return

        print(f"\n{'='*60}")
        print("BATCH ANALYSIS SUMMARY")
        print(f"{'='*60}")

        total_files = len(results)
        assessments = [r['analysis_summary']['plagiarism_assessment']['likelihood'] for r in results]

        high_count = assessments.count('HIGH')
        medium_count = assessments.count('MEDIUM')
        low_count = assessments.count('LOW')
        none_count = assessments.count('NONE')

        print(f"Total files analyzed: {total_files}")
        print(f"HIGH suspicion:       {high_count} ({high_count/total_files*100:.1f}%)")
        print(f"MEDIUM suspicion:     {medium_count} ({medium_count/total_files*100:.1f}%)")
        print(f"LOW suspicion:        {low_count} ({low_count/total_files*100:.1f}%)")
        print(f"NO suspicion:         {none_count} ({none_count/total_files*100:.1f}%)")

        # Show files with HIGH or MEDIUM suspicion
        suspicious_files = [r for r in results if r['analysis_summary']['plagiarism_assessment']['likelihood'] in ['HIGH', 'MEDIUM']]
        if suspicious_files:
            print(f"\n🚨 SUSPICIOUS FILES:")
            for result in suspicious_files:
                assessment = result['analysis_summary']['plagiarism_assessment']['likelihood']
                confidence = result['analysis_summary']['plagiarism_assessment']['confidence']
                print(f"   {result['query_song']} - {assessment} ({confidence})")
                if result['matched_works']:
                    top_match = result['matched_works'][0]
                    print(f"      → {top_match['work']} by {top_match['composer']}")

# Main function for production use
def analyze_test_files():
    """Main function to analyze all test files"""
    BASE_DIR = "/content/drive/MyDrive/MuseGuard"

    UNIFIED_MODEL_PATH = os.path.join(BASE_DIR, "best_unified_model.pth")
    REFERENCE_DB_PATH = os.path.join(BASE_DIR, "music_reference_base.pkl")

    # Check if files exist
    if not os.path.exists(UNIFIED_MODEL_PATH):
        print(f"ERROR: Model file not found: {UNIFIED_MODEL_PATH}")
        return

    if not os.path.exists(REFERENCE_DB_PATH):
        print(f"ERROR: Reference database not found: {REFERENCE_DB_PATH}")
        return

    # Initialize checker (no debug mode)
    checker = MusicPlagiarismChecker(
        unified_model_path=UNIFIED_MODEL_PATH,
        reference_db_path=REFERENCE_DB_PATH,
        debug=False
    )

    print("🎵 Music Plagiarism Detection System Ready")
    print("=" * 60)

    # Analyze all files in the base directory
    results = checker.analyze_batch(
        test_directory=BASE_DIR,
        output_file=os.path.join(BASE_DIR, "plagiarism_analysis_results.json")
    )

    return results

if __name__ == "__main__":
    analyze_test_files()

Loading OpenL3 model...
Loading unified contrastive model...
Loading reference database...
✅ Loaded 323 reference embeddings
🎵 Music Plagiarism Detection System Ready
Found 101 audio files to analyze...


Analyzing files:   1%|          | 1/101 [00:03<06:31,  3.91s/it]


📄 Bach_high_1.wav
   Assessment: MEDIUM
   Confidence: 0.36
   Matches: 1/4 segments
   Top match: Piano Quintet in A major by Schubert


Analyzing files:   2%|▏         | 2/101 [00:06<05:17,  3.21s/it]


📄 Bach_high_2.wav
   Assessment: MEDIUM
   Confidence: 0.35
   Matches: 1/4 segments
   Top match: Piano Quintet in A major by Schubert


...


📄 Schubert_low_2.wav
   Assessment: LOW
   Confidence: 0.32
   Matches: 3/4 segments
   Top match: Piano Quintet in A major by Schubert


Analyzing files:  99%|█████████▉| 100/101 [05:22<00:03,  3.33s/it]


📄 Schubert_low_3.wav
   Assessment: NONE
   Confidence: 0.17
   Matches: 3/4 segments
   Top match: Octet in E-flat major for Winds by Beethoven


Analyzing files: 100%|██████████| 101/101 [06:20<00:00,  3.77s/it]


📄 1759.wav
   Assessment: LOW
   Confidence: 0.41
   Matches: 2/39 segments
   Top match: Piano Sonata No 4 in E-flat major by Beethoven

Results saved to /content/drive/MyDrive/MuseGuard/plagiarism_analysis_results.json

BATCH ANALYSIS SUMMARY
Total files analyzed: 101
HIGH suspicion:       0 (0.0%)
MEDIUM suspicion:     18 (17.8%)
LOW suspicion:        41 (40.6%)
NO suspicion:         42 (41.6%)

🚨 SUSPICIOUS FILES:
   Bach_high_1.wav - MEDIUM (0.36)
      → Piano Quintet in A major by Schubert
   Bach_high_2.wav - MEDIUM (0.35)
      → Piano Quintet in A major by Schubert
   Bach_medium_3.wav - MEDIUM (0.36)
      → Octet in E-flat major for Winds by Beethoven
   Beethoven_medium_3.wav - MEDIUM (0.39)
      → Piano Quintet in A major by Schubert
   Brahms_high_2.wav - MEDIUM (0.35)
      → Octet in E-flat major for Winds by Beethoven
   Cambini_high_1.wav - MEDIUM (0.37)
      → Octet in E-flat major for Winds by Beethoven
   Cambini_medium_1.wav - MEDIUM (0.37)
      → Octet in E-